In [1]:
%run ./utils.ipynb
import gc

# STEP 1 READ THE DATA
read_dataset: takes path to JSON file that has sentences, _.EXR, _.TOP, _.TOP_DECOUPLED
and returns them as pandas.Series

In [ ]:
sentences, parsed_tree, structured_sentence, decoupled_structured_sentence = read_dataset("./PIZZA_train.json")

In [ ]:
# don't need them so free the data
del parsed_tree
del decoupled_structured_sentence


# STEP 2 : preprocess data
##### What we should take into consideration? 
1- Word Normalization  
2- Word Tokenization  
Why we won't use Sentence segmentation?  
It's useless, orders are one sentence question no clear punctuation exist

In [ ]:
# NORMALIZATION
normalized_sentence = pre_text_normalization(sentences.copy())

In [ ]:
# tokenization
vocab, tokenized_sentences = tokenization(normalized_sentence)


In [6]:
# checkpoint: Normalization and tokenization of sentences
vocab_as_series = pd.Series(list(vocab))
vocab_as_series.to_csv("vocab.csv",index=False)
tokenized_sentences.to_csv("tokenized_sentences.csv",index=False)

In [7]:
del vocab
del tokenized_sentences

# STEP 3: Parse data and extract labels

In [ ]:

pizza_orders, drink_orders, none_words = extract_pizza_drinks(structured_sentence.copy())

none_words = none_words.drop_duplicates()

none_words = pre_text_normalization(none_words)

none_words = none_words.reset_index(drop=True)

nones, _ = tokenization(none_words)


In [32]:
pizza_nodes, drink_nodes = extract_nodes(pizza_orders,drink_orders)
pizza_nodes, drink_nodes = clean_extracted_nodes(pizza_nodes, drink_nodes)
number, size, none, topping , quantity, style = pizza_nodes
drink_type, container_type, volume = drink_nodes

In [38]:

labels = [number, size, none, topping, quantity, style, drink_type, container_type, volume]
csv_file_names = ["number", "size", "none","topping","quantity","style","drink_type","container_type","volume"]
# merge nones that came from inside the PIZZAORDER, DRINKORDER and what u got from outside them
none_vocab = nones
for label, csv in zip(labels, csv_file_names):
    vocab, _ = tokenization(label)
    if csv == "none":
        vocab.update(none_vocab)
    vocab = pd.Series(list(vocab))
    vocab.to_csv(f"./labels/{csv}.csv", index=False)


In [39]:
del vocab
del labels
del csv_file_names
del pizza_nodes
del pizza_orders
del drink_nodes
del drink_orders

# STEP 4: Encode The tokens and label them

In [3]:
vocab, vocab_encoder, label_encoder = create_labeled_vocab(None)

In [4]:
# made sure that the encoding is correct
print(vocab_encoder.categories_[0][([vocab[vocab["tokens"] == "pizza"].loc[vocab[vocab["tokens"] == "pizza"].index[0],"encoded_tokens"]])])
print(label_encoder.categories_[0][([vocab[vocab["tokens"] == "pizza"].loc[vocab[vocab["tokens"] == "pizza"].index[0],"encoded_labels"]])])


['pizza']
['pizza']


In [5]:
tokenized_sentences = pd.read_csv("tokenized_sentences.csv")

C:\Users\abdo_\AppData\Local\Temp\ipykernel_19216\1794801712.py:1: DtypeWarning: Columns (14,15,16,17,18,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  tokenized_sentences = pd.read_csv("tokenized_sentences.csv")


In [6]:
convertor = conversions(vocab)

In [7]:
tokens_as_ids = tokenized_sentences.map(convertor.word2id)


In [8]:
labels_as_ids = tokenized_sentences.map(convertor.word2labels)

In [10]:
# from here on screw Pandas we only work with numpy, and tensors
tokens_ids_as_numpy = tokens_as_ids.to_numpy()
tokens_labels_as_numpy = labels_as_ids.to_numpy()

In [11]:
del tokenized_sentences
del tokens_as_ids
del labels_as_ids
gc.collect()

20

In [12]:
labels_as_numpy_fixed = np.array(tokens_labels_as_numpy.tolist(), dtype=np.uint8)


In [14]:
# check i did encode correctly:
encode_test = tokens_ids_as_numpy[0][~np.isnan(tokens_ids_as_numpy[0])] 
series = vocab[vocab["tokens"] == vocab_encoder.categories_[0][int(tokens_ids_as_numpy[0][3])] ]

index = series.index[0]

series1, series2 = series.loc[index,"encoded_labels"] , series.loc[index,"encoded_tokens"] 

print(vocab_encoder.categories_[0][series2])
print(label_encoder.categories_[0][series1])
# print(len(tokenized_sentences.loc[0]) == len(tokenized_sentences.loc[0]))


one
number


In [15]:
# the memory will be bad when we transform the numpy to tensor
# we need to split them and save them on disk so we can load batches when we train
tokens_batches = np.array_split(tokens_ids_as_numpy,10)
labels_batches = np.array_split(labels_as_numpy_fixed,10)
del tokens_ids_as_numpy
del tokens_labels_as_numpy
gc.collect()


482

In [ ]:
for i, batch in enumerate(tokens_batches):
    tensor_batch = torch.from_numpy(batch).type(torch.float32)
    torch.save(tensor_batch,f"./tokens_tensors/tokens_batch_{i}.pt")
for i, batch in enumerate(labels_batches):
    tensor_batch = torch.from_numpy(batch).type(torch.float32)
    torch.save(tensor_batch,f"./labels_tensors/labels_batch_{i}.pt")
del tensor_batch
del tokens_batches
del labels_batches

# STEP 5: MODEL

In [5]:
# determine what to run my tensors on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# my one hot encoders DETERMINSTIC ON WHAT CRITERIA I WILL TRAIN ON
input_size = vocab.shape[0]
# this is a parameter 
hidden_size = 300
# output : num of classes
num_classes = 11
# num of trials (epochs)
epochs = 10
# Batch size = ? 
batch_size = 10
# learning_rate
lr = 0.01

# num_layers in RNN default is 1 (increasing layers improve result but worsen the time)

In [6]:

# What is the model i will use
model = RNN(input_size,num_classes,hidden_size)
# loss criteria here i use CEloss
loss_criterion = nn.CrossEntropyLoss()
# stochastic gradient descent
optimizer = torch.optim.SGD(model.parameters(),lr)
# they say decaying learning rate is better than fixed one so i will use learning rate scheduler
lambdalr = lambda epoch: epoch / 10
scheduler = lr_scheduler.LambdaLR(optimizer,lambdalr)

In [ ]:

for epoch in range(1):
    
    for i in range(1):
        tokens = torch.load(f"./tokens_tensors/tokens_batch_{i}.pt",weights_only=True).type(torch.int)
        labels = torch.load(f"./labels_tensors/labels_batch_{i}.pt",weights_only=True).type(torch.int)

        dataset = SimpleDataset(tokens,labels)
        data = DataLoader(dataset,batch_size=batch_size)
        
        del tokens


        model.train()
        
        for input_tensors, label_tensors in data:
            
            input_tensors = input_tensors.to(device)
            
            label_tensors = label_tensors.to(device)
           
            out_tensor = model(input_tensors)
            
            mask_condition = label_tensors > 0

            label_tensors = (label_tensors * mask_condition).type(torch.float32)

            loss = loss_criterion(out_tensor,label_tensors)

            optimizer.zero_grad()

            loss.backward()
            
            optimizer.step()
        
    scheduler.step()
    print(f"epoch {epoch}:, loss ={loss.item()}")

KeyboardInterrupt: 